In [14]:
import h5py
import pathlib
import numpy as np
import tqdm

In [2]:
pairs_files = list(map(lambda x: h5py.File(x, 'r'), pathlib.Path('./').rglob('./**/*pairs*.h5')))
pairs_files

[<HDF5 file "protein.pairs.g500.fromorg.virushost.v10.5.h5" (mode r)>,
 <HDF5 file "protein.pairs.viruses.v10.5.h5" (mode r)>,
 <HDF5 file "protein.pairs.hosts.v10.5.h5" (mode r)>]

In [3]:
full_seq_file = h5py.File('./full.protein.sequences.v10.5.h5', 'r')

In [1]:
# !rm protein.sequences.v10.5.h5

In [4]:
rel = []
for f in pairs_files:
    p = f['pairs'][:]
    rel.extend(p // (2**32))
    rel.extend(p % (2**32))
rel.extend(np.random.randint(full_seq_file['is_virus'].shape[0], size=100000))
rel = np.array(rel)
rel = np.unique(rel)
rel.sort()

In [5]:
f = h5py.File('protein.sequences.v10.5.h5', 'w')

In [6]:
f.create_dataset('is_virus', 
                 compression='gzip', compression_opts=7, 
                 data=full_seq_file['is_virus'][:])

<HDF5 dataset "is_virus": shape (9660620,), type "|b1">

In [7]:
seq_len = np.array(full_seq_file['seq_len'][:]).copy().astype(np.int32)
seq_len_rel = seq_len[rel]
seq_len[:] = 0
seq_len[rel] = seq_len_rel
f.create_dataset('seq_len', 
                 compression='gzip', compression_opts=7,
                 data=seq_len)

<HDF5 dataset "seq_len": shape (9660620,), type "<i4">

In [8]:
seq_start = f['seq_len'][:]
seq_start[1:] = seq_start[:-1]
seq_start[0] = 0
seq_start = seq_start.cumsum()
f.create_dataset('seq_start', 
                 compression='gzip', compression_opts=7, 
                 data=seq_start)

<HDF5 dataset "seq_start": shape (9660620,), type "<i8">

In [12]:
str_all = full_seq_file['seqs']
ss = full_seq_file['seq_start']
sl = full_seq_file['seq_len']
def get(i): return str_all[ss[i]:ss[i]+sl[i]]

In [16]:
rel_str = [get(i) for i in tqdm.tqdm(rel)]
rel_str = np.concatenate(rel_str)
rel_str.shape

100%|██████████| 115247/115247 [00:39<00:00, 2952.77it/s]


(45309396,)

In [17]:
f.create_dataset('seqs', 
                 compression='gzip', compression_opts=7,
                 data=rel_str)

<HDF5 dataset "seqs": shape (45309396,), type "|S1">

In [ ]:
str_all1 = f['seqs']
ss1 = f['seq_start']
sl1 = f['seq_len']
def get1(i): return str_all1[ss1[i]:ss1[i]+sl1[i]]
for i in tqdm.tqdm(rel):
    assert (get(i) == get1(i)).all()

 64%|██████▎   | 73318/115247 [01:03<00:43, 965.91it/s] 

In [ ]:
del f

In [ ]:
!ls -lh